## Introduction

Explain your chosen topic and why it is important for your field.

在这个项目里，我选择了图论以及图结构作为主题，并且想阐述图论是如何在传统金融行业（比如商业银行）中进行辅助应用的。
面对现今金融时代爆发增长的企业需求，以及瞬息万变的市场环境，原来单点突破进行风险控制的方法已经远远不够。急需通过科技手段，分析企业风险、提高工作效率、来降低信贷成本。传统金融行业对企业信用风险进行识别和检测时，使用的大多还是传统方法：现场调查、基于规则的预测等方式。

而我们知道商业银行内部有较完善的客户数据以及交易数据，基于这些数据以及对结构化及半结构化数据的挖掘，通过知识图谱在商业银行的垂直领域（比如股权关系，，，）进行深耕，能够挖掘及展现出出很多我们无法在普通关系性数据库发现的细节。如此一来，通过编织出一张张知识图谱，我们可以辅助金融行业的高风险管理及控制。从而提升商业银行对于企业经营风险的预判能力，及早预测，规避风险。

In this project, I chose graph theory and graph structure as the theme, and would like to explain how graph theory can be applied in the traditional financial industry (e.g. commercial banks).
In the face of the explosive growth of enterprise demand in the current financial era and the rapidly changing market environment, the original single-point breakthrough approach to risk control is far from being sufficient. There is an urgent need to analyze enterprise risk, improve work efficiency and reduce credit costs through scientific and technological means. When the traditional financial industry identifies and detects enterprise credit risk, most of the traditional methods are still used: on-site investigation, rule-based prediction and so on.

As we know that commercial banks have better customer data and transaction data, based on these data and the mining of structured and semi-structured data, through the knowledge graph in the vertical areas of commercial banks (e.g., equity relationship, etc.), we can dig and show a lot of details that we can not be found in ordinary relational databases. In this way, by weaving a knowledge graph, we can assist in high risk management and control in the financial industry. In this way, we can enhance the ability of commercial banks to anticipate business risks, make early predictions, and avoid risks.

## Literature Review

### 讨论知识图谱的进化发展历程（简短）



### 讨论金融系统（商业银行）里进行数据分析的复杂性和重要性（简短）



### 讨论知识图谱在金融行业的应用（重点）

1. fraud detection


2. risk assessment and management


3. investment analysis / regulatory compliance



## Example : Simple Shareholding Graph System

### Description



### Data source 

The data was from gihub project [Nebula Graph Shareholding project](https://github.com/wey-gu/nebula-shareholding-example/tree/main). And the author of the project says that the data was a faker data which will simulates the real situation and does not cause information leakage. So I borrowed the author's data generator code and made changes to complete the data acquisition for this project.

There are various kinds of relationships when we checking companies' shareholding breakthrough, here let's simplify it with only two kind of entities: person and corp, and with following relationship types.

- `person` can hold a `corp` in {share} %

- `person` can be relative with another `person`

- `corp` can hold another `corp` in {share} %

- `corp` can be a branch of another `corp`

- `person` can be as a role of a `corp`


### Code

#### Data preparation

因为原始数据集是放进图数据库直接存储，可以在图数据库中直接设置tag和edge。而我在example中需要自己实现graph类，所以需要对原始数据集进行预处理。

#### Data generators

由于github上的sample数据集数量较大，在简单的图可视化过程中可能会造成可视化过于复杂难懂的问题。所以我们通过以下代码简化数据集大小及复杂度，便于后面的展示。

In [ ]:
import csv
import pandas as pd
from faker import Faker
from random import randint

PERSON_COUNT = 8
CORP_COUNT = 6
PERSON_SHAREHOLD_COUNT = 15
CORP_REL_COUNT = 10
CORP_SHAREHOLD_COUNT = 10
PERSON_REL_COUNT = 4
#PERSON_ROLE_COUNT = 50

WRITE_BATCH = 10

faker = Faker()
Faker.seed(1234)  # Correct way to seed

def csv_writer(file_path, row_count, row_generator):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file, delimiter=',', quotechar="'", quoting=csv.QUOTE_MINIMAL)
        csv_buffer = []
        for row in range(row_count):
            csv_buffer.append(row_generator())
            if len(csv_buffer) >= WRITE_BATCH:
                writer.writerows(csv_buffer)
                csv_buffer = []
        if csv_buffer:
            writer.writerows(csv_buffer)

# Generate Persons and Corporations using Faker
def generate_person_data():
    data = {'person_id': ['p_' + str(i) for i in range(PERSON_COUNT)], 'person_name': [faker.name() for _ in range(PERSON_COUNT)]}
    return pd.DataFrame(data)

def generate_corp_data():
    data = {'corp_id': ['c_' + str(i) for i in range(CORP_COUNT)], 'corp_name': [faker.company() for _ in range(CORP_COUNT)]}
    return pd.DataFrame(data)


# Save persons and corporations to CSV
generate_person_data().to_csv('data/person.csv', index=False)
generate_corp_data().to_csv('data/corp.csv', index=False)

# Relationship generators
def person_share_generator():
    return ('p_' + str(randint(0, PERSON_COUNT - 1)), 'c_' + str(randint(0, CORP_COUNT - 1)), randint(0, 100))

def person_rel_generator():
    return ('p_' + str(randint(0, PERSON_COUNT - 1)), 'p_' + str(randint(0, PERSON_COUNT - 1)), randint(1, 3))

def corp_rel_generator():
    return ('c_' + str(randint(0, CORP_COUNT - 1)), 'c_' + str(randint(0, CORP_COUNT - 1)))

def corp_share_generator():
    return ('c_' + str(randint(0, CORP_COUNT - 1)), 'c_' + str(randint(0, CORP_COUNT - 1)), randint(0, 100))
'''
def person_corp_role_generator():
    job_title = faker.job().replace(',', '')
    return ('p_' + str(randint(0, PERSON_COUNT - 1)), 'c_' + str(randint(0, CORP_COUNT - 1)), job_title)
'''
# Writing the relationships to CSV
csv_writer('data/person_corp_share.csv', PERSON_SHAREHOLD_COUNT, person_share_generator)
csv_writer('data/person_rel.csv', PERSON_REL_COUNT, person_rel_generator)
csv_writer('data/corp_rel.csv', CORP_REL_COUNT, corp_rel_generator)
csv_writer('data/corp_share.csv', CORP_SHAREHOLD_COUNT, corp_share_generator)
#csv_writer('data/person_corp_role.csv', PERSON_ROLE_COUNT, person_corp_role_generator)

#### Import Data

而在这一步我们将data generator出来的csv数据导入，生成dataframe，便于后续往graph class传参数。

In [ ]:
import pandas as pd

# [1] corp.csv
# define the corporation id and name from the csv file
path1 = './data/corp.csv'
# load the csv into a DataFrame, and specify the column names
corp_df = pd.read_csv(path1)
print(corp_df.head(6))

# [2] person.csv
# define the person id and name from the csv file
path2 = './data/person.csv'
person_df = pd.read_csv(path2)
#print(person_df.head(6))

# [3] person_rel.csv
# define the relationship degree of person id1 and id2 from the csv file
path3 = './data/person_rel.csv'
column_names = ['person_id1', 'person_id2','rel_degree']
person_rel_df = pd.read_csv(path3, header=None, names=column_names)
#print(person_rel_df.head(6))

# [4] corp_rel.csv
# define the relationship of corporation id1 and id2 from the csv file
path4 = './data/corp_rel.csv'
column_names = ['corp_id1', 'corp_id2']
corp_rel_df = pd.read_csv(path4, header=None, names=column_names)
#print(corp_rel_df.head(6))

# [5] corp_share.csv
# define the holdsharing of corporation id1 to id2 from the csv file
path5 = './data/corp_share.csv'
column_names = ['corp_id1', 'corp_id2','share_per']
corp_share_df = pd.read_csv(path5, header=None, names=column_names)
#print(corp_share_df.head(6))

# [6] person_corp_share.csv
# define the holdsharing of person id to the corporation id from the csv file
path6 = './data/person_corp_share.csv'
column_names = ['person_id', 'corp_id','share_per']
person_corp_share_df = pd.read_csv(path6, header=None, names=column_names)
print(person_corp_share_df.head(6))

'''
# [7] person_corp_role.csv
# define the role of person id in the corporation id from the csv file
path7 = './data/person_corp_role.csv'
column_names = ['person_id', 'corp_id','role']
person_corp_role_df = pd.read_csv(path7, header=None, names=column_names)
print(person_corp_role_df.head(6))
'''

#### Define the SelfdefinedGraph Class

The SelfdefinedGraph Class
Each of the bullt point here represents an element of this class.
- `__init__(self)`. This 

- `add_vertex(self, vertex, vertex_type)`.

- `add_edge(self, from_vertex, to_vertex, relationship_type, weight=None)`

- `remove_vertex(self, vertex)`.

- `remove_edge(self, from_vertex, to_vertex)`.

- `update_edge(self, from_vertex, to_vertex, new_weight)`.

- `print_adjacency_list(self)`.

- ``

In [ ]:
# Python code to illustrate the example
import networkx as nx
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.sparse import coo_matrix

class SelfdefinedGraph:
    def __init__(self):
        self.adjacency_list = {}
        self.G = nx.DiGraph()

    def add_vertex(self, vertex, vertex_type):
        if vertex not in self.adjacency_list:
            self.adjacency_list[vertex] = {'type': vertex_type, 'edges': []}
            self.G.add_node(vertex, type=vertex_type)

    def add_edge(self, from_vertex, to_vertex, relationship_type, weight=None):
        if from_vertex in self.adjacency_list:
            self.adjacency_list[from_vertex]['edges'].append({
                'to': to_vertex,
                'type': relationship_type,
                'weight': weight
            })
            self.G.add_edge(from_vertex, to_vertex, weight=weight, type=relationship_type)

    def remove_vertex(self, vertex):
        if vertex in self.adjacency_list:
            # Remove all edges to this vertex
            for v in self.adjacency_list:
                self.adjacency_list[v]['edges'] = [e for e in self.adjacency_list[v]['edges'] if e['to'] != vertex]
            # Remove the vertex itself
            del self.adjacency_list[vertex]

    def remove_edge(self, from_vertex, to_vertex):
        if from_vertex in self.adjacency_list:
            self.adjacency_list[from_vertex]['edges'] = [
                e for e in self.adjacency_list[from_vertex]['edges'] if e['to'] != to_vertex
            ]

    def update_edge(self, from_vertex, to_vertex, new_weight):
        if from_vertex in self.adjacency_list:
            for edge in self.adjacency_list[from_vertex]['edges']:
                if edge['to'] == to_vertex:
                    edge['weight'] = new_weight

    def query_vertices_by_type(self, vertex_type):
        # Return vertices of a specific type
        return [vertex for vertex, properties in self.adjacency_list.items() if properties['type'] == vertex_type]

    def query_edges_by_type(self, from_vertex, edge_type):
        # Return edges of a specific type from a given vertex
        if from_vertex in self.adjacency_list:
            return [edge for edge in self.adjacency_list[from_vertex]['edges'] if edge['type'] == edge_type]

    def query_edges_by_weight(self, from_vertex, min_weight, max_weight):
        # Return edges within a certain weight range from a given vertex
        if from_vertex in self.adjacency_list:
            return [edge for edge in self.adjacency_list[from_vertex]['edges'] if min_weight <= edge['weight'] <= max_weight]

    def print_adjacency_list(self):
        adjacency_list = self.adjacency_list
        return adjacency_list

在这里我需要对graph类生成的adjacency_list进行可视化，就有了下面的visualize_graph function。

In [ ]:
def visualize_graph(adjacency_list):
    G = nx.DiGraph()
    # Add nodes with different styles if needed
    for node, properties in adjacency_list.items():
        G.add_node(node, type=properties['type'])

    # Add edges with different colors based on the relationship type
    edge_colors = []
    for node, properties in adjacency_list.items():
        for edge in properties['edges']:
            G.add_edge(node, edge['to'], weight=edge['weight'])
            if edge['type'] == 'holds':
                edge_colors.append('blue')  # Blue for financial relationships
            elif edge['type'] == 'relative':
                edge_colors.append('green')  # Green for family relationships

    pos = nx.spring_layout(G)  # Positions for all nodes
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color=edge_colors, width=2, node_size=20)
    #labels = nx.get_edge_attributes(G, 'weight')
    #nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    plt.title('Graph Visualization of Relationships')
    plt.show()

In [ ]:
# Initialize graph
graph = SelfdefinedGraph()

# Add persons and corps as vertices
for _, row in person_df.iterrows():
    graph.add_vertex(row['person_id'], 'person')
for _, row in corp_df.iterrows():
    graph.add_vertex(row['corp_id'], 'corp')

# Add relationship as edges
for _, row in person_corp_share_df.iterrows():
    graph.add_edge(row['person_id'], row['corp_id'], 'holds', row['share_per'])

for _, row in person_rel_df.iterrows():
    graph.add_edge(row['person_id1'], row['person_id2'], 'relative', row['rel_degree'])

for _, row in corp_rel_df.iterrows():
    graph.add_edge(row['corp_id1'], row['corp_id2'], 'branch')

for _, row in corp_share_df.iterrows():
    graph.add_edge(row['corp_id1'], row['corp_id2'], 'holds',row['share_per'])

'''
for _, row in person_corp_role_df.iterrows():
    graph.add_edge(row['person_id'], row['corp_id'], 'roles', row['role'])
'''

In [ ]:
graph.print_adjacency_list()

In [ ]:
#print(graph.query_vertices_by_type('person'))
print(graph.query_edges_by_type('p_5', 'relative'))

In [ ]:
'''
import networkx as nx
import matplotlib.pyplot as plt

G_test = nx.DiGraph()
G_test.add_node("A",type = 'person')
G_test.add_node("B",type = 'corp')
G_test.add_edge("A", "B")
pos_test = nx.circular_layout(G_test)
nx.draw(G_test, pos_test, with_labels=True)
plt.show()
'''

In [ ]:
adjacency_list = graph.adjacency_list
#visualize_graph(adjacency_list=adjacency_list)

In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import community as community_louvain

# Create a graph
G = nx.karate_club_graph()
# Compute the best partition
partition = community_louvain.best_partition(G)
# Draw the graph
pos = nx.spring_layout(G)  # Position nodes using Fruchterman-Reingold force-directed algorithm
cmap = plt.get_cmap('viridis')  # Color map for coloring nodes
# Color the nodes according to their partition
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=40,
                       cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()


AttributeError: module 'community' has no attribute 'best_partition'

## Conclusion

这种图结构出来的知识图谱可以运用到金融行业里，比如说进行客户分群，精准提升金融行业

## References {.unnumbered}

::: {#refs}
:::
